In [1]:
import sys  
sys.path.insert(0, '/Users/pratikaher/DGL/GRecSy/')

from random import sample
import dgl
import torch
import torch.nn.functional as F
import pandas as pd
from utils import set_random_seed, collate_molgraphs, load_model, collate_movie_graphs, train_test_split_by_time, _split_data
import numpy as np
import torch
import torch.nn as nn
import argparse
from configure import get_exp_configure
import scipy.sparse as sp
from torch.utils.data import DataLoader

from dgllife.utils import EarlyStopping, Meter
from model import compute_loss

df_rating = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'],  engine='python')
df_user = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/users.dat', sep='::', header=None, names=['user_id', 'gender', 'age', 'occupation', 'zipcode'],  engine='python')
df_movie = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genre'],  engine='python')

df_temp = df_rating.merge(df_movie, left_on='movie_id', right_on='movie_id', how='left')
df_final = df_temp.merge(df_user, left_on='user_id', right_on='user_id', how='left')

df_final = df_final[["user_id","movie_id","rating","age"]]

/Users/pratikaher/DGL/graph-rec/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_final.tail()

,user_id,movie_id,rating,age
1000204,6040,1091,1,25
1000205,6040,1094,5,25
1000206,6040,562,5,25
1000207,6040,1096,4,25
1000208,6040,1097,4,25


In [3]:
hetero_graph = dgl.heterograph({('user', 'rates', 'movie'): (df_final['user_id'].to_numpy(), df_final['movie_id'].to_numpy())})

In [4]:
hetero_graph.nodes['user'].data['age'] = torch.randn(6041, 1).float()
hetero_graph.nodes['movie'].data['duration'] = torch.randn(3953, 1).float()
hetero_graph.edges['rates'].data['rating'] = torch.randn(1000209, 1)

In [5]:
def construct_negative_graph(graph, k, etype):
    utype, _, vtype = etype
    src, dst = graph.edges(etype=etype)
    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.num_nodes(vtype), (len(src) * k,))
    return dgl.heterograph(
        {etype: (neg_src, neg_dst)},
        num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

In [6]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, etype):
        h = self.sage(g, x)
        return self.pred(g, h, etype), self.pred(neg_g, h, etype)

In [7]:
from dgllife.model.gnn import MPNNGNN
import dgl.function as fn

class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        # h contains the node representations for each node type computed from
        # the GNN defined in the previous section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']
        
class MPNNPredictor(nn.Module):
    def __init__(self,
                 node_in_feats,
                 edge_in_feats,
                 node_out_feats=64,
                 edge_hidden_feats=128,
                 n_tasks=1,
                 num_step_message_passing=6):
        super(MPNNPredictor, self).__init__()

        self.gnn = MPNNGNN(node_in_feats=node_in_feats,
                           node_out_feats=node_out_feats,
                           edge_in_feats=edge_in_feats,
                           edge_hidden_feats=edge_hidden_feats,
                           num_step_message_passing=num_step_message_passing)
        self.predictor = HeteroDotProductPredictor()
    
    def forward(self, g, node_feats, edge_feats, node_subgraph_negative):
        node_feats = self.gnn(g, node_feats, edge_feats)
        return self.predictor(g, node_feats, ('user', 'rates', 'movie')), self.predictor(node_subgraph_negative, node_feats)
    
    def get_repr(self, node_features):
        """Returns the embedded representation given block made from sampling neighboring nodes."""
        
        node_feats = self.gnn(g, node_feats, edge_feats)
        return node_feats
        

In [8]:
mpnn_model = MPNNPredictor(hetero_graph.ndata['age']['user'].shape[1], hetero_graph.edata['rating'].shape[1])

In [9]:
hetero_graph.ndata['age']['user'].shape[1]

1

In [10]:
hetero_graph.edata['rating']

tensor([[-0.5269],
        [-2.1338],
        [-1.0997],
        ...,
        [ 0.3458],
        [ 0.6460],
        [-0.0843]])

In [20]:
hetero_graph

Graph(num_nodes={'movie': 3953, 'user': 6041},
      num_edges={('user', 'rates', 'movie'): 1000209},
      metagraph=[('user', 'movie', 'rates')])

In [24]:
hetero_graph.edges['rates']

EdgeSpace(data={'rating': tensor([[ 0.0550],
        [ 0.8098],
        [-0.6900],
        ...,
        [ 0.8672],
        [-1.1030],
        [ 0.4908]])})

In [11]:
user_feats = hetero_graph.nodes['user'].data['age']
item_feats = hetero_graph.edges['rates'].data['rating']
negative_graph = construct_negative_graph(hetero_graph, 5, ('user', 'rates', 'movie'))
pos_score, neg_score = mpnn_model(hetero_graph, user_feats, item_feats, negative_graph)

RuntimeError: Expected hidden size (1, 3953, 64), got [1, 6041, 64]

In [56]:
k = 5
user_feats = hetero_graph.nodes['user'].data['age']
item_feats = hetero_graph.nodes['movie'].data['duration']
node_features = {'user': user_feats, 'item': item_feats}
opt = torch.optim.Adam(model.parameters())
for epoch in range(10):
    negative_graph = construct_negative_graph(hetero_graph, k, ('user', 'rates', 'movie'))
    pos_score, neg_score = mpnn_model(hetero_graph, user_feats, item_feats, negative_graph)
    loss = compute_loss(pos_score, neg_score)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

RuntimeError: random_ expects 'from' to be less than 'to', but got from=0 >= to=0

In [55]:
hetero_graph.ndata

defaultdict(<class 'dict'>, {'duration': {'movie': tensor([], size=(0, 1))}, '_ID': {'movie': tensor([], dtype=torch.int64), 'user': tensor([], dtype=torch.int64)}, 'age': {'user': tensor([], size=(0, 1))}})

In [54]:
hetero_graph = dgl.in_subgraph(hetero_graph, {'dst_type': 0}, relabel_nodes=True)